<a href="https://colab.research.google.com/github/ggratani/embryo_segmentation/blob/main/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask_cors

In [4]:
!pip install flask_ngrok

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import os
from flask import send_from_directory
import cv2
from flask_ngrok import run_with_ngrok

In [ ]:
def format_label(clase, x, y, width, height, image_width, image_height):
    x_center = x / image_width
    y_center = y / image_height
    norm_width = width / image_width
    norm_height = height / image_height
    return f"{clase} {x_center} {y_center} {norm_width} {norm_height}\n".format(clase, x_center, y_center, norm_width, norm_height)


In [ ]:
config = {
    # "DEBUG": True,
    "CACHE_TYPE": "SimpleCache",
    "CACHE_DEFAULT_TIMEOUT": 120
}

app = Flask(__name__)
CORS(run_with_ngrok(app))

# Configura la carpeta "images" como una carpeta estática
app.static_folder = r'C:\Users\gasto\Documents\React\embryoxite\backend_flask'

app.config.from_mapping(config)

@app.route('/alive')
def alive():
    print("alive")
    return jsonify({"mensaje": "Alive"})

In [ ]:
# Configuracion ruta deteccion

UPLOAD_FOLDER_DETECTION = r'C:\Users\gasto\Documents\React\embryoxite\backend_flask\detection'

# Carpeta para las imágenes
app.config['IMAGE_UPLOAD_FOLDER_DETECTION'] = os.path.join(UPLOAD_FOLDER_DETECTION, 'images')
# Carpeta para las anotaciones
app.config['LABEL_UPLOAD_FOLDER_DETECTION'] = os.path.join(UPLOAD_FOLDER_DETECTION, 'labels')

# Configuracion ruta segmentation

UPLOAD_FOLDER_SEGMENTATION = r'segmentation'

# Carpeta para las imágenes
app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'] = os.path.join(UPLOAD_FOLDER_SEGMENTATION, 'images')
# Carpeta para las anotaciones
app.config['LABEL_UPLOAD_FOLDER_SEGMENTATION'] = os.path.join(UPLOAD_FOLDER_SEGMENTATION, 'labels')


In [ ]:
@app.route('/save_image_detection', methods=['POST'])
def save_image():

    if 'image' not in request.files:
        return jsonify({"mensaje": "No se encontró el archivo en la solicitud"}), 400

    image = request.files['image']
    labels = request.files['labels'].read().decode('utf-8')

    print(labels)

    if image.filename == '':
        return jsonify({"mensaje": "No se seleccionó ningún archivo"}), 400

    if image and labels:
        # Verificar que la extensión del archivo sea válida (puedes personalizar esto según tus necesidades)
        allowed_extensions = {'png', 'jpg', 'jpeg', 'gif'}
        if '.' in image.filename and image.filename.rsplit('.', 1)[1].lower() in allowed_extensions:
            # Guardar el archivo en la carpeta de carga
            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_DETECTION'], image.filename)
            image.save(filename)

            # Supongamos que 'annotations' contiene las anotaciones en formato YOLO
            with open(os.path.join(app.config['LABEL_UPLOAD_FOLDER_DETECTION'], image.filename.rsplit('.', 1)[0] + ".txt"), 'w') as file:
                info = labels.split()
                x = (float(info[1]) + float(info[3]))/2
                y = (float(info[2]) + float(info[4]))/2

                width = float(info[3])-float(info[1])
                height = float(info[4])-float(info[2])
                file.write(format_label(info[0], x, y, float(width), float(height), float(info[5]), float(info[6])))

            return jsonify({"mensaje": "Imagen guardada con éxito"}), 200
        else:
            return jsonify({"mensaje": "Extensión de archivo no permitida"}), 400


In [ ]:
@app.route('/save_image_segmentation', methods=['POST'])
def save_image_segmentation():

    if 'image' not in request.files:
        return jsonify({"mensaje": "No se encontró el archivo en la solicitud"}), 400

    image = request.files['image']
    image2 = request.files['image2']

    print(image)

    if image.filename == '':
        return jsonify({"mensaje": "No se seleccionó ningún archivo"}), 400

    if image:
        # Verificar que la extensión del archivo sea válida (puedes personalizar esto según tus necesidades)
        allowed_extensions = {'png', 'jpg', 'jpeg', 'gif'}
        if '.' in image.filename and image.filename.rsplit('.', 1)[1].lower() in allowed_extensions:
            # Guardar el archivo en la carpeta de carga
            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'], image.filename)
            image.save(filename)

            # Decodifica la matriz de bytes en una imagen de OpenCV en escala de grises
            imagen_opencv = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

            # Define el umbral deseado (ajusta este valor según tus necesidades)
            umbral = 200

            # Aplica el umbral con cv2.THRESH_BINARY_INV
            _, imagen_umbralizada = cv2.threshold(imagen_opencv, umbral, 255, cv2.THRESH_BINARY)

            # Guarda la imagen umbralizada
            cv2.imwrite(os.path.join(app.config['LABEL_UPLOAD_FOLDER_SEGMENTATION'], image.filename), imagen_umbralizada)

            filename = os.path.join(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'], image.filename)
            image2.save(filename)

            return jsonify({"mensaje": "Imagen guardada con éxito"}), 200
        else:
            return jsonify({"mensaje": "Extensión de archivo no permitida"}), 400

In [ ]:
@app.route('/get_images_detection', methods=['GET'])
def get_images_detection():
    # Obtener la lista de nombres de archivo en el directorio de imágenes
    image_files = os.listdir(app.config['IMAGE_UPLOAD_FOLDER_DETECTION'])

    # Filtrar la lista para incluir solo archivos de imagen (por extensión)
    allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_files = [file for file in image_files if any(file.endswith(ext) for ext in allowed_extensions)]

    # Devolver la lista de nombres de archivo como respuesta JSON
    return jsonify({'images': image_files})

In [ ]:
@app.route('/get_images_segmentation', methods=['GET'])
def get_images_segmentation():
    # Obtener la lista de nombres de archivo en el directorio de imágenes
    image_files = os.listdir(app.config['IMAGE_UPLOAD_FOLDER_SEGMENTATION'])

    # Filtrar la lista para incluir solo archivos de imagen (por extensión)
    allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_files = [file for file in image_files if any(file.endswith(ext) for ext in allowed_extensions)]

    # Devolver la lista de nombres de archivo como respuesta JSON
    return jsonify({'images': image_files})

# Ruta para servir las imágenes estáticas
@app.route('/images/<path:filename>')
def serve_image(filename):
    print(filename)
    return send_from_directory(app.static_folder, filename)

In [12]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4143-34-133-40-112.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on /save_image_segmentation [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-12-9a41de880238>", line 109, in save_image_segmentation
    image.save(filename)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/datastructures/file_storage.py", line 125, in save
    dst = open(dst, "wb")
FileNotFoundError: [Errno 2] No such file or directory: 'segmentation/images/imagen_169785412119

<FileStorage: 'imagen_1697854121198.png' ('image/png')>


INFO:werkzeug:127.0.0.1 - - [21/Oct/2023 02:11:07] "POST /save_image_segmentation HTTP/1.1" 200 -


<FileStorage: 'imagen_1697854266183.png' ('image/png')>


In [14]:
!git remote add origin https://github.com/ggratani/embryo_segmentation.git

fatal: not a git repository (or any of the parent directories): .git
